# Python3 Fluency Workbook  

## Logging, Errors and Exceptions

The purpose of this notebook is to help you get comfortable with logging, debugging generating documentation in Python3

# Setup

In [ ]:
import logging

In [78]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Logging

Usage and best practices for using Pythons native `logging` module.

## Loggers, Levels, Formatters and Handlers

To get started with Python's logging module you need to understand loggers, handlers and formatters:

> **Loggers** have **levels** (how severe the issue is) and **handlers** (that tell it where to put the information) which have **formatters** (which tell it how to format/display the information).

### Get or create loggers

Python's native logging module is hierarchical. There is a default root logger then when you add custom loggers you are adding "children" to the root logger.

**NOTE:** The root logger (`logging.root` - top of the logging hierarchy) is called when methods like logging.debug() is used. 

* Default log level: is WARN, so every log with lower level (for example via logging.info("info")) will be ignored. 
* Default handler:  its default handler will be created the first time a log with a level greater than WARN is logged. 

**Using the root logger directly or indirectly via methods like logging.debug() is generally not recommended. Instead, create your own loggers.**

In [80]:
test1_logger = logging.getLogger()

In [81]:
test1_logger #=>Defaults to the root logger because we didn't specify a logger name

<RootLogger root (WARNING)>

In [82]:
# Get or create a logger named "test2"
test2_logger = logging.getLogger("test2") #=>Defulats to child of root logger because we specifies a name

In [87]:
print(test2_logger)
print(test2_logger.parent)
assert test1_logger.parent == logging.root # the parent is the root logger

<Logger test2 (WARNING)>
<RootLogger root (WARNING)>


AssertionError: 

Hierarchal loggers use dot notation

In [89]:
t = logging.getLogger("t"); t #=>Create logger, t

<Logger t (WARNING)>

In [90]:
t.parent #=>t's parent is root

<RootLogger root (WARNING)>

In [91]:
a = logging.getLogger("t.a");a #=>Create logger a as sublogger of t

<Logger t.a (WARNING)>

In [92]:
a.parent #=>a's parent is t

<Logger t (WARNING)>

In [93]:
a.parent.parent #=>a's parents parent == t's parent == root

<RootLogger root (WARNING)>

Inspect your logger's default settings (log level and handlers)

In [84]:
print('Handlers for {} logger? {}'.format(test1_logger.name, test1_logger.hasHandlers()))
print('Handlers for {} logger? {}'.format(test2_logger.name, test2_logger.hasHandlers()))

Handlers for root logger? True
Handlers for test2 logger? True


In [73]:
print('Level for {} logger? {}'.format(test1_logger.name, test1_logger.getEffectiveLevel()))
print('Level for {} logger? {}'.format(test2_logger.name, test2_logger.getEffectiveLevel()))

Level for root logger? 30
Level for test2 logger? 30


In [74]:
#Note that if you log a message > WARN using root logger it will create a handler
print(test1_logger.hasHandlers())
logging.error("test error message for root logger")
print(test1_logger.hasHandlers())

ERROR:root:test error message for root logger


True
True


### Create handlers

Each handler has:

* A formatter which adds context information to a log.

* A log level that filters out logs whose levels are inferior (ie. a log handler with INFO level will not handle DEBUG logs)

For example, a log formatter like this

`"%(asctime)s — %(name)s — %(levelname)s — %(funcName)s:%(lineno)d — %(message)s"`

will become

`2018-02-07 19:47:41,864 - a.b.c - WARNING - <module>:1 - your log message`

In [27]:
console_handler = logging.StreamHandler()
console_handler

<StreamHandler stderr (NOTSET)>

In [28]:
file_handler = logging.FileHandler("filename")
file_handler

<FileHandler /Users/elias/jupyter-notebooks/python3_fluency/filename (NOTSET)>

Now we can add the handlers to any of our loggers

### Create log messages

If the log level is lower than logger level, the log will be ignored

Basically, when you create a logger you say "show me anything above this level of severity, ignore everything else".

Here are the 6 log levels

| 6 Log Levels |
|---|
| `NOTSET=0` |
| `DEBUG=10` |
| `INFO=20` |
| `WARN=30` |
| `ERROR=40` |
| `CRITICAL=50` |

## A Practical Example w/ Best Practices: Logging

**Best practices**

* Configure the root logger but never use it in your code—e.g., never call a function like logging.info()

* To use the logging, make sure to create a new logger by using `logging.getLogger(logger name)`. I usually use  __name__ as the logger name, but anything can be used, as long as it is consistent.


*Note: Logs can get...quite long. So you may want to use classes like `RotatingFileHandler` to rotate the file for you automatically when the file reaches a size limit or do it everyday. You may also want to look into tools like Sentry or Airbrake for really long logs like from WebApps.*

### Example 1: Logging w/ default handler and formatter

In [101]:
logger = logging.getLogger(__name__)

print(logger)
print('hasHandlers? {}'.format(logger.hasHandlers()))
print('getEffectiveLevel? {}'.format(logger.getEffectiveLevel()))

<Logger __main__ (WARNING)>
hasHandlers? True
getEffectiveLevel? 30


In [105]:
# Debug and info should be ignored because our log level is WARNING
logger.debug('my DEBUG message')
logger.info('my INFO message')
logger.warning('my WARNING message')
logger.error('my ERROR message')
logger.critical('my CRITICAL message')

ERROR:__main__:my ERROR message
CRITICAL:__main__:my CRITICAL message


### Example 2: Logging w/ custom handler and formatter

In [106]:
# Get or create a logger
logger = logging.getLogger(__name__)  

# Set log level
logger.setLevel(logging.DEBUG)

# Define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

# Add file handler to logger
logger.addHandler(file_handler)

In [107]:
logger.debug('A debug message')
logger.info('An info message')
logger.warning('Something is not right.')
logger.error('A Major error has happened.')
logger.critical('Fatal error. Cannot continue')

DEBUG:__main__:A debug message
INFO:__main__:An info message
ERROR:__main__:A Major error has happened.
CRITICAL:__main__:Fatal error. Cannot continue


We can checkout the log file that was just created.

Notice the format for the log file vs the console output is different, that is because its being handled by a different handler. The default console handler and the log file handler we just created.

In [108]:
!cat logfile.log

2019-11-17 13:04:07,452 : DEBUG : __main__ : A debug message
2019-11-17 13:04:07,453 : INFO : __main__ : An info message
2019-11-17 13:04:07,454 : WARNING : __main__ : Something is not right.
2019-11-17 13:04:07,458 : ERROR : __main__ : A Major error has happened.
2019-11-17 13:04:07,460 : CRITICAL : __main__ : Fatal error. Cannot continue


### Example 3: Logging an exception/error (div by 0)

In [110]:
import logging

# Create or get the logger
logger = logging.getLogger(__name__)  

# set log level
logger.setLevel(logging.INFO)

def divide(x, y):
    try:
        out = x / y
    except ZeroDivisionError:
        logger.exception("Division by zero problem")
    else:
        return out

In [112]:
divide(10,0)

ERROR:__main__:Division by zero problem
Traceback (most recent call last):
  File "<ipython-input-110-0b146286ffa0>", line 11, in divide
    out = x / y
ZeroDivisionError: division by zero


# Errors and Exceptions

Raising and handling default and custom exceptions

**How try statements work:**

First the try clause (the statement(s) between the try and except keywords) is executed.

If no exception occurs, the except clause is skipped and execution of the try statement is finished.

If an exception occurs during execution of the try clause, the rest of the clause is skipped. Then if its type matches the exception named after the except keyword, the except clause is executed, and then execution continues after the try statement.

If an exception occurs which does not match the exception named in the except clause, it is passed on to outer try statements; if no handler is found, it is an **unhandled exception** and execution stops with a message as shown above.

Finally, the finally (if its there) is always executed and can be useful for cleanup tasks like closing sockets, etc.

### Basic try-except

In [114]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")

Please enter a number: we
Oops!  That was no valid number.  Try again...
Please enter a number: 12


### Raise exception

In [117]:
# Raise an exception with custom message
raise ValueError("custom message")

ValueError: custom message

### Define custom exceptions

In [ ]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class InputError(Error):
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class TransitionError(Error):
    """Raised when an operation attempts a state transition that's not
    allowed.

    Attributes:
        previous -- state at beginning of transition
        next -- attempted new state
        message -- explanation of why the specific transition is not allowed
    """

    def __init__(self, previous, next, message):
        self.previous = previous
        self.next = next
        self.message = message

### Exception inheritance

In [118]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

In [119]:
for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
C
D


The last except doesn't need an exception name(s). Its just treated as a wildcard.

In [120]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


### Finally is always executed

In [121]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

### Else is executed IFF try works

In [128]:
# Else is executed
try:
    print("trying worked")
except OSError:
    print('inside OSError')
else:
    print("inside else")
finally:
    print("inside finally")

trying worked
inside else
inside finally


In [127]:
# Else isn't executed
try:
    raise OSError("some OS error")
except (OSError, RuntimeError, TypeError):
    print('inside OSError')
else:
    print("inside else")
finally:
    print("inside finally")

inside OSError
inside finally
